In [1]:
import duckdb
import pandas as pd

In [2]:
# Подключение к базе данных DuckDB
db_path = '../db/edfitter.duckdb'
conn = duckdb.connect(database=db_path, read_only=True)

In [3]:
courses_df = conn.execute("SELECT * FROM main.courses").fetchdf()
feedback_df = conn.execute("SELECT * FROM main.feedback").fetchdf()
users_df = conn.execute("SELECT * FROM main.users").fetchdf()

In [4]:
# Объединение данных
# 1. Объединяем `courses` и `feedback` по `session_id`, задаем суффиксы, чтобы избежать "_x" и "_y"
courses_feedback_df = courses_df.merge(feedback_df, on='session_id', how='left', suffixes=('', '_feedback'))

In [5]:
# 2. Добавляем данные из `users` по `user_id`, избегая "_x" и "_y"
final_df = courses_feedback_df.merge(users_df, left_on='user_id', right_on='id', how='left', suffixes=('', '_user'))

In [6]:
# Объединение столбцов user_id и id в один столбец
final_df['user_id'] = final_df['user_id'].fillna(final_df['id'])
# Удаление лишней колонки 'id', которая больше не нужна
final_df = final_df.drop(columns=['id'], errors='ignore')
# Удаление столбца `Уровень обучения`
final_df = final_df.drop(columns=['level'], errors='ignore')

In [7]:
# Переименование и упорядочивание столбцов
final_df = final_df.rename(columns={
    'user_id': 'user_id',
    'session_id': 'session_id',
    'username': 'Имя пользователя',
    'group_number': 'Номер группы (только для студентов)',
    'role': 'Роль или направление',
    'message': 'Сообщение пользователя',
    'relevant_vacancy_skills': 'Требуемые навыки (релевантные)',
    'unrelated_vacancy_skills': 'Требуемые навыки (удаленные)',
    'additional_vacancy_skills': 'Требуемые навыки (добавленные)',
    'label': 'Уровень обучения',
    'discipline': 'Дисциплина',
    'department': 'Кафедра',
    'faculty': 'Факультет кафедры',
    'campus': 'Кампус',
    'audience': 'Охват аудитории',
    'format': 'Формат изучения',
    'course_number': 'Курс обучения',
    'relevance': 'Релевантность курса запросу',
    'feedback_comment': 'Отзыв пользователя о сервисе',
    'utility': 'Полезность',
    'popularity': 'Востребованность',
    'comfort': 'Удобство'
})

# Установка порядка столбцов в соответствии с `rename`
column_order = [
    'user_id', 'session_id', 'Имя пользователя', 'Номер группы (только для студентов)', 
    'Роль или направление', 'Сообщение пользователя', 'Требуемые навыки (релевантные)', 
    'Требуемые навыки (удаленные)', 'Требуемые навыки (добавленные)', 'Уровень обучения', 'Дисциплина', 
    'Кафедра', 'Факультет кафедры', 'Кампус', 'Охват аудитории', 
    'Формат изучения', 'Курс обучения', 'Релевантность курса запросу', 
    'Отзыв пользователя о сервисе', 'Полезность', 'Востребованность', 'Удобство'
]

# Перестановка столбцов
final_df = final_df[column_order]
final_df.head(5) 
# Сохранение результата в CSV

,user_id,session_id,Имя пользователя,Номер группы (только для студентов),Роль или направление,Сообщение пользователя,Требуемые навыки (релевантные),Требуемые навыки (удаленные),Требуемые навыки (добавленные),Уровень обучения,...,Факультет кафедры,Кампус,Охват аудитории,Формат изучения,Курс обучения,Релевантность курса запросу,Отзыв пользователя о сервисе,Полезность,Востребованность,Удобство
0,ca7576b82a75220d,4be1816022125a65,Тест,463463463,Руководитель образовательной программы,логист,Транспортная логистика; Автомобильные грузопер...,Работа в команде; Деловое общение; Логистика; ...,123; вппк63; рркеркенкн кенекнкен,Бакалавриат,...,Высшая школа бизнеса,Москва,для своего кампуса,без онлайн-курса,не указан,5,вапвапвапв,7,5,3
1,ca7576b82a75220d,4be1816022125a65,Тест,463463463,Руководитель образовательной программы,логист,Транспортная логистика; Автомобильные грузопер...,Работа в команде; Деловое общение; Логистика; ...,123; вппк63; рркеркенкн кенекнкен,Бакалавриат,...,Факультет менеджмента,НИУ ВШЭ - Нижний Новгород,для своего кампуса,без онлайн-курса,не указан,2,вапвапвапв,7,5,3
2,ca7576b82a75220d,4be1816022125a65,Тест,463463463,Руководитель образовательной программы,логист,Транспортная логистика; Автомобильные грузопер...,Работа в команде; Деловое общение; Логистика; ...,123; вппк63; рркеркенкн кенекнкен,Магистратура,...,Факультет Санкт-Петербургская школа экономики ...,НИУ ВШЭ - Санкт-Петербург,для всех кампусов НИУ ВШЭ,без онлайн-курса,None,6,вапвапвапв,7,5,3
3,ca7576b82a75220d,4be1816022125a65,Тест,463463463,Руководитель образовательной программы,логист,Транспортная логистика; Автомобильные грузопер...,Работа в команде; Деловое общение; Логистика; ...,123; вппк63; рркеркенкн кенекнкен,Магистратура,...,Высшая школа бизнеса,Москва,для своего кампуса,без онлайн-курса,не указан,6,вапвапвапв,7,5,3
4,ca7576b82a75220d,4be1816022125a65,Тест,463463463,Руководитель образовательной программы,логист,Транспортная логистика; Автомобильные грузопер...,Работа в команде; Деловое общение; Логистика; ...,123; вппк63; рркеркенкн кенекнкен,"Бакалавриат, Специалитет, Магистратура, Аспира...",...,Факультет Санкт-Петербургская школа гуманитарн...,НИУ ВШЭ - Санкт-Петербург,для своего кампуса,без онлайн-курса,не указан,2,вапвапвапв,7,5,3


In [8]:
final_df.to_csv('../db/edfitter_data.csv', index=False)

In [9]:
# Закрытие соединения
conn.close()